In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('ppp_transformed.csv')

In [3]:
df = pd.read_csv('ppp_transformed.csv')
df_drop = df.drop(columns=df.iloc[:,41:67])
df_drop = df_drop.drop(columns=['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'BorrowerName', 'BorrowerCity', 'BorrowerZip',
                      'InitialApprovalAmount', 'FranchiseName', 'ServicingLenderName', 'ServicingLenderZip', 'BusinessAgeDescription', 'NAICSCode',
                      'Industry', 'ProjectCity', 'TOTAL_PROCEED', 'UTILITIES_PROCEED_pct', 'HEALTH_CARE_PROCEED_pct',
                      'DEBT_INTEREST_PROCEED_pct'])

object_cols = [col for col in df_drop.columns if df_drop[col].dtype == object]
new_dummies = pd.get_dummies(df_drop[object_cols], drop_first=True, dtype=int)
df_new = pd.concat((df_drop.drop(columns=object_cols), new_dummies), axis=1)

# object_cols
df_new.shape

(961899, 219)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961899 entries, 0 to 961898
Data columns (total 68 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   LoanNumber                                  961899 non-null  int64  
 1   DateApproved                                961899 non-null  object 
 2   SBAOfficeCode                               961899 non-null  int64  
 3   ProcessingMethod                            961899 non-null  object 
 4   BorrowerName                                961892 non-null  object 
 5   BorrowerCity                                961899 non-null  object 
 6   BorrowerState                               961899 non-null  object 
 7   BorrowerZip                                 961899 non-null  object 
 8   LoanStatus                                  961899 non-null  object 
 9   Term                                        961899 non-null  int64  
 

In [5]:
# df = df.drop(columns=['ProcessingMethod_encoded','LoanStatus_encoded', 'RuralUrbanIndicator_encoded', 'HubzoneIndicator_encoded',
                    #    'LMIIndicator_encoded', 'BusinessAgeDescription_encoded', 'Race_encoded', 'Ethnicity_encoded'])

# drop previously encoded
df_drop = df.drop(columns=df.iloc[:,41:67])
df_drop = df_drop.drop(columns=['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'BorrowerName', 'BorrowerCity', 'BorrowerZip',
                      'InitialApprovalAmount', 'FranchiseName', 'ServicingLenderName', 'ServicingLenderZip', 'BusinessAgeDescription', 'NAICSCode',
                      'Industry', 'ProjectCity', 'TOTAL_PROCEED', 'UTILITIES_PROCEED_pct', 'HEALTH_CARE_PROCEED_pct',
                      'DEBT_INTEREST_PROCEED_pct'])
df_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961899 entries, 0 to 961898
Data columns (total 24 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ProcessingMethod               961899 non-null  object 
 1   BorrowerState                  961899 non-null  object 
 2   LoanStatus                     961899 non-null  object 
 3   Term                           961899 non-null  int64  
 4   CurrentApprovalAmount          961899 non-null  float64
 5   RuralUrbanIndicator            961899 non-null  object 
 6   HubzoneIndicator               961899 non-null  object 
 7   LMIIndicator                   961899 non-null  object 
 8   JobsReported                   961899 non-null  float64
 9   Race                           961899 non-null  object 
 10  Ethnicity                      961899 non-null  object 
 11  BusinessType                   961899 non-null  object 
 12  Gender                        

In [6]:
df_drop.BusinessType.value_counts()

BusinessType
Corporation                            415037
Limited  Liability Company(LLC)        259213
Subchapter S Corporation               174534
Non-Profit Organization                 55332
Partnership                             18014
Limited Liability Partnership           12627
Sole Proprietorship                     12176
Professional Association                 6344
Cooperative                              2414
501(c)3 – Non Profit                     1768
Non-Profit Childcare Center               899
501(c)6 – Non Profit Membership           764
Unknown                                   664
Employee Stock Ownership Plan(ESOP)       638
Self-Employed Individuals                 495
Trust                                     401
Housing Co-op                             152
Joint Venture                             129
Independent Contractors                   112
Single Member LLC                          77
Tribal Concerns                            63
Tenant in Common     

In [7]:
object_cols = [col for col in df_drop.columns if df_drop[col].dtype == object]
new_dummies = pd.get_dummies(df_drop[object_cols], drop_first=True, dtype=int)
df_new = pd.concat((df_drop.drop(columns=object_cols), new_dummies), axis=1)

# object_cols
df_new.shape

(961899, 219)

Models to try
- Logistic regression
- svm
- randomforrest
- bagging
- nn

In [8]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import classification_report

In [34]:
X = df_new.drop(columns=['isFraud'])
y = df_new['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# ros = RandomOverSampler()
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

In [39]:
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

In [40]:
print(classification_report(y_test, y_pred_lr, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      0.70      0.82    240453
       Fraud       0.00      0.27      0.00        22

    accuracy                           0.70    240475
   macro avg       0.50      0.48      0.41    240475
weighted avg       1.00      0.70      0.82    240475



In [41]:
from sklearn.linear_model import SGDClassifier

In [44]:
loss_list = ['hinge', 'log_loss']

for loss in loss_list:
    penalty =  'l2'
    if loss == 'log_loss':
        penalty = 'l1'
    print(f"\n{loss}")
    model = SGDClassifier(loss=loss, penalty=penalty)
    model.fit(X_train, y_train)
    y_pred_model = model.predict(X_test)
    print(classification_report(y_test, y_pred_model, target_names=['Not Fraud', 'Fraud']) + '\n')


hinge
              precision    recall  f1-score   support

   Not Fraud       1.00      0.28      0.43    240453
       Fraud       0.00      0.95      0.00        22

    accuracy                           0.28    240475
   macro avg       0.50      0.62      0.22    240475
weighted avg       1.00      0.28      0.43    240475



log_loss
              precision    recall  f1-score   support

   Not Fraud       1.00      0.74      0.85    240453
       Fraud       0.00      0.23      0.00        22

    accuracy                           0.74    240475
   macro avg       0.50      0.48      0.42    240475
weighted avg       1.00      0.74      0.85    240475



modified_huber
              precision    recall  f1-score   support

   Not Fraud       1.00      0.20      0.34    240453
       Fraud       0.00      1.00      0.00        22

    accuracy                           0.20    240475
   macro avg       0.50      0.60      0.17    240475
weighted avg       1.00      0.20      

c:\Users\melov\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\melov\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\melov\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00    240453
       Fraud       0.00      0.00      0.00        22

    accuracy                           1.00    240475
   macro avg       0.50      0.50      0.50    240475
weighted avg       1.00      1.00      1.00    240475



huber


c:\Users\melov\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


              precision    recall  f1-score   support

   Not Fraud       1.00      0.00      0.00    240453
       Fraud       0.00      1.00      0.00        22

    accuracy                           0.00    240475
   macro avg       0.50      0.50      0.00    240475
weighted avg       1.00      0.00      0.00    240475



epsilon_insensitive


c:\Users\melov\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\melov\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\melov\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\melov\AppData\Local\Programs\Python\

              precision    recall  f1-score   support

   Not Fraud       0.00      0.00      0.00    240453
       Fraud       0.00      1.00      0.00        22

    accuracy                           0.00    240475
   macro avg       0.00      0.50      0.00    240475
weighted avg       0.00      0.00      0.00    240475



squared_epsilon_insensitive


c:\Users\melov\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\melov\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00    240453
       Fraud       0.00      0.00      0.00        22

    accuracy                           1.00    240475
   macro avg       0.50      0.50      0.50    240475
weighted avg       1.00      1.00      1.00    240475




c:\Users\melov\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
svm_model = SGDClassifier(loss='log_loss', penalty='l1')
model.fit(X_train, y_train)
y_pred_model = model.predict(X_test)
print(classification_report(y_test, y_pred_model, target_names=['Not Fraud', 'Fraud']) + '\n')

MemoryError: Unable to allocate 4.57 GiB for an array with shape (1442714, 425) and data type float64

In [56]:
from sklearn.svm import SVC

In [57]:
svc_model = SVC()
svc_model.fit(X_train, y_train)

MemoryError: Unable to allocate 4.57 GiB for an array with shape (1442714, 425) and data type float64

In [ ]:
y_pred_svc = svc_model.predict(y_test)
print(classification_report(y_test, y_pred_svc, target_names=['Not Fraud', 'Fraud']))

In [52]:
from xgboost import XGBRFClassifier

In [55]:
xgb_model = XGBRFClassifier()
X_train_xgb = np.array(X_train)
xgb_model.fit(X_train_xgb, y_train)

MemoryError: Unable to allocate 4.57 GiB for an array with shape (1442714, 425) and data type float64

In [ ]:
print(classification_report(y_test, y_pred_model, target_names=['Not Fraud', 'Fraud']))